In [21]:
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
import os
from requests_html import HTMLSession

sesh = HTMLSession()

A large base cleaning function

In [22]:
# okay, dataframe isn't correct, going to use a weblink in the meantime
review_link = 'https://www.ign.com/articles/2009/08/26/batman-arkham-asylum-collectors-edition-review'
review_sess = sesh.get(review_link)
review_soup = BeautifulSoup(review_sess.html.html)

In [128]:
review_script = review_soup.find('script',{'id':'__NEXT_DATA__'}).string
mark1 = re.search('paginatedHtmlPage', review_script)

mod_date_place = re.search('"modifiedDate"', review_script)
mod_date = review_script[mod_date_place.end()+2:mod_date_place.end()+26]

pub_date_place = re.search('"publishDate"', review_script)
pub_date = review_script[pub_date_place.end()+2:pub_date_place.end()+26]

tag_slug_place = re.search('"tagSlugs"', review_script)
cat_slug_place = re.search('"categorySlugs"', review_script)
eve_slug_place = re.search('"eventSlugs"', review_script) # don't care about these, but a good stop point

# can almost guarantee a review is a game now
tags = review_script[tag_slug_place.end():cat_slug_place.start()]
categories = review_script[cat_slug_place.end():eve_slug_place.start()]

# cut down on how much script I have to look at and clean
pag_html_place = re.search('paginatedHtmlPage', review_script)
inter_script = review_script[pag_html_place.end():]
script_start_place = re.search('":\["', inter_script)
script_end_place = re.search('"\],"', inter_script)
review_body = inter_script[script_start_place.end():script_end_place.start()]

# some other information that I may want
auth_dets_place = re.search('"authorSocialDetails"', review_script)
auth_dets_stop = re.search('"__typename":"Article"', review_script)
auth_dets = review_script[auth_dets_place.end():auth_dets_stop.start()]

name_place_start = re.search('"name":', auth_dets)
name_place_end = re.search(',"thumbnailUrl"', auth_dets)

auth_name = auth_dets[name_place_start.end():name_place_end.start()]

# review summary and score/stuff
verd_start_place = re.search('"review":{', review_script)
verd_end_place = re.search('"Review"}', review_script)
verdict_full = review_script[verd_start_place.end():verd_end_place.start()]

# could try to put this from the review_script instead of the verdict piece
verd_text_start = re.search('"comment":', verdict_full)
verd_text_end = re.search(',"score":', verdict_full)
verd_score_text = re.search(',"scoreText"', verdict_full)
verd_choice = re.search('"editorsChoice"', verdict_full)

review_summary = verdict_full[verd_text_start.end():verd_text_end.start()]
review_score = verdict_full[verd_text_end.end():verd_score_text.start()]
review_score_text = verdict_full[verd_score_text.end():verd_choice.start()]

# how about getting the name of the game being reviewed!
# ,"names":{"name":"Batman: Arkham Asylum","short":null,"alt":[],"__typename":"Names"},"__typename":
# start with title, occurs earlier, more specific tag
title_start = re.search('"title":', review_script)
title_end = re.search(',"url"', review_script)
title = review_script[title_start.end():title_end.start()]

# "productName":"Batman: Arkham Asylum (PS3)","imageUrl":"","vendor":"Gamestop","canonicalVendor":"Gamestop","__typename":"Commerce"},"objectRegions":[{"id":1444929,"name":"Batman: Arkham Asylum","shortName"

game_start = re.search('"productName":', review_script)
game_end = re.search('"imageUrl":"",', review_script)
game_name = review_script[game_start.end():game_end.start()]

# print(game_start, game_end)
print(title)
print(game_name)
# print(review_body)
# print(review_summary)
# print(review_score)
# print(review_score_text)
# print()
# print(verdict_full)
# print(auth_dets)
# print(auth_name)
# print(review_script[auth_dets_place.start():])
# print(inter_script[script_start_place.end():script_end_place.start()])
# print(tags)
# print(categories)
# print(pub_date)       
print(review_script)

<re.Match object; span=(25807, 25821), match='"productName":'> <re.Match object; span=(25851, 25865), match='"imageUrl":"",'>
"Batman: Arkham Asylum Collector's Edition Review"
"Batman: Arkham Asylum (PS3)",
{"props":{"pageProps":{"page":{"canonical":"/articles/2009/08/26/batman-arkham-asylum-collectors-edition-review","id":"4faba26e9e4e5021868ba6bb","modifiedDate":"2012-05-10T11:11:42+0000","platformTag":null,"publishDate":"2009-08-26T00:46:00+0000","publishState":"published","promoSummary":"It's not often that I'm conflicted about a game. Don't get me wrong, review scores are in the eye of the beholder so I'm sure there are plenty of times that you've read something I have written and not agreed with me. I mean that it's rare for me to play a game during a demo and not have a solid idea of what the final product is going to be. I played bits of Batman: Arkham Asylum a few times before I finally got a copy to review, and I always found myself coming away with mixed feelings. I enjoyed

In [ ]:
def split_script_to_parts(soup):
    
    review_script = review_soup.find('script',{'id':'__NEXT_DATA__'}).string
    mark1 = re.search('paginatedHtmlPage', review_script)

    mod_date_place = re.search('"modifiedDate"', review_script)
    mod_date = review_script[mod_date_place.end()+2:mod_date_place.end()+26]

    pub_date_place = re.search('"publishDate"', review_script)
    pub_date = review_script[pub_date_place.end()+2:pub_date_place.end()+26]

    tag_slug_place = re.search('"tagSlugs"', review_script)
    cat_slug_place = re.search('"categorySlugs"', review_script)
    eve_slug_place = re.search('"eventSlugs"', review_script) # don't care about these, but a good stop point

    # can almost guarantee a review is a game now
    tags = review_script[tag_slug_place.end():cat_slug_place.start()]
    categories = review_script[cat_slug_place.end():eve_slug_place.start()]
    
    # cut down on how much script I have to look at and clean
    pag_html_place = re.search('paginatedHtmlPage', review_script)
    inter_script = review_script[pag_html_place.end():]
    script_start_place = re.search('":\["', inter_script)
    script_end_place = re.search('"\],"', inter_script)
    review_body = inter_script[script_start_place.end():script_end_place.start()]

    # some other information that I may want
    auth_dets_place = re.search('"authorSocialDetails"', review_script)
    auth_dets_stop = re.search('"__typename":"Article"', review_script)
    auth_dets = review_script[auth_dets_place.end():auth_dets_stop.start()]

    # review summary and score/stuff
    verd_start_place = re.search('"review":{', review_script)
    verd_end_place = re.search('"Review"}', review_script)
    verdict_full = review_script[verd_start_place.end():verd_end_place.start()]

    # could try to put this from the review_script instead of the verdict piece
    verd_text_start = re.search('"comment":', verdict_full)
    verd_text_end = re.search(',"score":', verdict_full)
    verd_score_text = re.search(',"scoreText"', verdict_full)
    verd_choice = re.search('"editorsChoice"', verdict_full)

    review_summary = verdict_full[verd_text_start.end():verd_text_end.start()]
    review_score = verdict_full[verd_text_end.end():verd_score_text.start()]
    review_score_text = verdict_full[verd_score_text.end():verd_choice.start()]
    
    review_info = {'review_summary': review_summary, 'review_score': review_score, 
                   'review_score_text': review_score_text, 'review_body': review_body, 
                   'tags': tags, 'categories': categories, 'modified_date': mod_date, 
                   'published_date': pub_date, 'author_details': auth_dets}
    
    # try to find release date of game, for instance of historical game
    
    return review_info

In [116]:
def text_cleaner(text):
    
    final_text = re.sub('\\\\u003e', '', text)
    final_text = re.sub('\\\\u003cbr /', '', final_text)
    final_text = re.sub('\\\\u003csection', '', final_text)
    final_text = re.sub('\\\\u003c/section', '', final_text)
    final_text = re.sub('\\\\u003cp', '', final_text)
    final_text = re.sub('\\\\u003cstrong', '', final_text)
    final_text = re.sub('\\\\u003c/strong', '', final_text)
    final_text = re.sub('\\\\u003c/p', '', final_text)
    final_text = re.sub('data-transform', '', final_text)
    final_text = re.sub('=', '', final_text)
    final_text = re.sub('cellpadding', '', final_text)
    final_text = re.sub('cellspacing', '', final_text)
    final_text = re.sub('table width', '', final_text)
    final_text = re.sub('class=\\\\"article-page\\\\"', '', final_text)
    final_text = re.sub('class\\\\"article-page\\\\"', '', final_text)
    final_text = re.sub('class\\\\"aligncenter\\\\"', '', final_text)
    final_text = re.sub('\\\\"mobile-ad-break\\\\"', '', final_text)
    final_text = re.sub('\\\\u003c/a', '', final_text)
    final_text = re.sub('\\\\u003ca', '', final_text)
    final_text = re.sub('\\\\u003c!', '', final_text)
    final_text = re.sub('","', '', final_text)
#     final_text = re.sub('\\\\"', '', final_text)
#     final_text = re.sub('/section', '', final_text)

    # consider getting rid of the later half of the string, as it largely duplicates the review

#     rough_cut = re.search('paginatedH', final_text)
#     final_text = final_text[:rough_cut.start()]
    
    start_iter = re.finditer('start image div', final_text)
    end_iter = re.finditer('end image div', final_text)
    image_starts = []
    image_ends = []
    
    for iter1 in start_iter:
        image_starts.append(iter1.start() )
    for iter2 in end_iter:
        image_ends.append(iter2.end() )

    for i in range (0, len(image_starts) ):
        if i == 0:
            mid_text = final_text[:image_starts[i]] 
            
        elif i >= 1 and i < len(image_starts)-1:
            mid_text += final_text[image_ends[i-1]:image_starts[i]]
            
        elif i == len(image_starts)-1:
            mid_text += final_text[image_ends[i-1]:image_starts[i]] + final_text[image_ends[i]:]
    
    final_text = mid_text
    
    # commented out because need to examine the href a bit more
    # seems like it might be fixed by using review_body
    href_starts = []
    href_ends = []
    href_start_iter = re.finditer('href', final_text)
    href_end_iter = re.finditer('html', final_text)
    for iter1 in href_start_iter:
        href_starts.append(iter1.start() )
    for iter2 in href_end_iter:
        href_ends.append(iter2.end() )
        
    print('num link starts ', len(href_starts))
    print('num link ends ', len(href_ends))
    
    for i in range(0, len(href_starts) ):
        if i == 0:
            mid_text = final_text[:href_starts[i]]
        elif i >= 1 and i < len(href_starts)-1:
            mid_text += final_text[href_ends[i-1]:href_starts[i]]
        elif i == len(href_starts)-1:
            mid_text += final_text[href_ends[i-1]:href_starts[i]] + final_text[href_ends[i]:]
    final_text = mid_text
    
    # getting rid of image links
    image_ref_starts = []
    image_ref_ends = []
    image_starts_iter = re.finditer('\\\\"\\\\u003cimg', final_text)
    image_ends_iter = re.finditer('.jpg', final_text) # may want other versions of this for different images
    
    for iter1 in image_starts_iter:
        image_ref_starts.append(iter1.start() )
    for iter2 in image_ends_iter:
        image_ref_ends.append(iter2.end() )
    
    print('num image starts ', len(image_ref_starts) )
    print('num image ends', len(image_ref_ends) )
    
    for i in range(0, len(image_ref_starts) ):
        if i == 0:
            mid_text = final_text[:image_ref_starts[i]]
        elif i >= 1 and i < len(image_ref_starts)-1:
            mid_text += final_text[image_ref_ends[i-1]:image_ref_starts[i]]
        elif i == len(image_ref_starts)-1:
            mid_text += final_text[image_ref_ends[i-1]:image_ref_starts[i]] + final_text[image_ref_ends[i]:]
    final_text = mid_text
    
    return final_text

In [117]:
# ignore_text_cut = review_script.string[mark1.end():]
# print(text_cleaner(review_script.string))

print(text_cleaner(review_body))

num link starts  5
num link ends  5
num image starts  2
num image ends 2
 It's not often that I'm conflicted about a game. Don't get me wrong, review scores are in the eye of the beholder so I'm sure there are plenty of times that you've read something I have written and not agreed with me. I mean that it's rare for me to play a game during a demo and not have a solid idea of what the final product is going to be. I played bits of Batman: Arkham Asylum a few times before I finally got a copy to review, and I always found myself coming away with mixed feelings. I enjoyed the combat and attacking from the shadows, but the pacing seemed off and Batman's movements seemed stiff. Could Rocksteady really pull everything together to deliver on one of DC Comics' biggest franchises?     After having beaten the game, I can tell you the answer is most certainly \"Yes.\"    \u003c\"460\" border\"1\" \"0\" \"5\"\u003ctr bgcolor\"#f1f1f3\"\u003ctd Collector's Edition Content Batman: Arkham Asylum's a

In [ ]:
# time to try the cleaner on a different 

In [4]:
raw_df = pd.read_csv('ign_data/unedited_review_html.csv', sep='`')

In [6]:
print(raw_df.shape)
raw_df.head()

(5642, 3)


,Unnamed: 0,link,unfiltered_html
0,0,https://www.ign.com/articles/2017/07/21/girls-...,"<script id=""__NEXT_DATA__"" type=""application/j..."
1,1,https://www.ign.com/articles/2017/07/22/comic-...,"<script id=""__NEXT_DATA__"" type=""application/j..."
2,2,https://www.ign.com/articles/2017/05/18/12-mon...,"<script id=""__NEXT_DATA__"" type=""application/j..."
3,3,https://www.ign.com/articles/2017/07/22/my-her...,"<script id=""__NEXT_DATA__"" type=""application/j..."
4,4,https://www.ign.com/articles/2017/07/21/death-...,"<script id=""__NEXT_DATA__"" type=""application/j..."


In [19]:
print(raw_df['link'].loc[0])

https://www.ign.com/articles/2017/07/21/girls-trip-review-2


In [20]:
bat_df = raw_df.loc[
    raw_df['link']=='https://www.ign.com/articles/2009/08/26/batman-arkham-asylum-collectors-edition-review']
bat_df.head()

,Unnamed: 0,link,unfiltered_html


In [ ]:
# okay, dataframe isn't correct, going to use a weblink in the meantime
